In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

from api_keys import g_key 
gmaps.configure(api_key=g_key)

from googleplaces import GooglePlaces, types, lang
google_places = GooglePlaces(g_key)



In [2]:
weather_data = pd.read_csv('Output/weather_data.csv')
weather_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Araouane,18.90,-3.53,87.10,13,0,8.01,ML,1603667539
1,1,Leningradskiy,69.38,178.42,24.51,90,68,5.41,RU,1603667539
2,2,Kruisfontein,-34.00,24.73,61.00,96,100,1.01,ZA,1603667539
3,3,Hobart,-42.88,147.33,57.00,54,75,11.41,AU,1603667540
4,4,Bluff,-46.60,168.33,53.01,63,71,5.99,NZ,1603667370
...,...,...,...,...,...,...,...,...,...,...
550,550,Aktau,43.65,51.20,55.40,93,90,2.24,KZ,1603667626
551,551,Palembang,-2.92,104.75,73.40,100,20,3.89,ID,1603667626
552,552,Gari,59.43,62.35,25.16,98,100,2.75,RU,1603667626
553,553,Toamasina,-18.17,49.38,69.80,94,40,2.95,MG,1603667626


In [3]:
locations = weather_data[['Lat', 'Lng']]
humidity = weather_data['Humidity']
fig = gmaps.figure(center=(51.5, -0.1), zoom_level=1.4)
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity))
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
city_name = []
country = []
lat = []
lng = []
hotel_name = []

ideal_weather = weather_data.query('`Max Temp` > 70 and `Max Temp` < 80 and `Wind Speed` < 10 and Cloudiness == 0')
for index, row in ideal_weather.iterrows():
    query_result = google_places.nearby_search( 
        lat_lng ={'lat': row['Lat'], 'lng': row['Lng']}, 
        radius = 5000,         
        types =[types.TYPE_LODGING])      
    if len(query_result.places) > 0:
        city_name.append(row['City'])
        country.append(row['Country'])
        for place in query_result.places: 
            hotel_name.append(place.name)
            lat.append(float(place.geo_location['lat']))
            lng.append(float(place.geo_location['lng'])) 
            break

hotel_data = {
    "City": city_name,
    "Lat":lat, 
    "Lng":lng,    
    "Country":country,
    "Hotel":hotel_name, 
    }  

hotel_df = pd.DataFrame(hotel_data)

In [21]:
#figy = gmaps.figure(center=(51.5, -0.1), zoom_level=1.4)
locations = hotel_df[['Lat', 'Lng']]
concat_locs = []
for index, row in hotel_df.iterrows():
    concat_locs.append(row['Hotel'] + " ," + row['City'] + " ," + row['Country'])
marker_layer = gmaps.marker_layer(locations,info_box_content=concat_locs)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))